In [255]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import random
import time

In [66]:
recipes = 'https://cooking.nytimes.com/search?filters%5Bmeal_types%5D%5B%5D=dinner&q=&page=1'
response = requests.get(recipes)
response.status_code
page = response.text
soup = BeautifulSoup(page, "lxml")



<!DOCTYPE html>
<html lang="en-US">
 <head>
  <title>
   Search Thousands of Recipes - NYT Cooking
  </title>
  <script>
   window.googletag = window.googletag || {};
      window.googletag.cmd = window.googletag.cmd || [];
  </script>
  <script>
   if (typeof window.nyt_et === 'function') {
        window.nyt_et('init', 'https://a.et.nytimes.com/', 'nytcooking', {
            subject: 'page',
            assetUrl: (document.querySelector('link[rel=canonical]') || {}).href,
            assetId: (document.querySelector('meta[name=articleid]') || {}).content,
            nyt_uri: (document.querySelector('meta[name=nyt_uri]') || {}).content,
            url: location.href,
            referrer: document.referrer || void 0,
            client_tz_offset: (new Date).getTimezoneOffset(),
        });
    }
  </script>
  <script>
   var NYTD=NYTD||{};NYTD.Abra=function(t){"use strict";function e(t){var e=r[t];return e&&e[1]||null}function n(t,e){if(t){var n,r,o=e[0],i=e[1],c=0,u=0;if(1!==i.leng

In [67]:
# scrape all recipe links from search page and add to list
#take first 40 pages, seems to date back to ~2014 when they were promoting digital content

all_recipes = []
for p in range(1,41,1):
    url = "https://cooking.nytimes.com/search?filters%5Bmeal_types%5D%5B%5D=dinner&q=&page="+str(p)
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    for i in soup.find_all('article', {'class': 'card recipe-card'}):
        link_string = i.find('a')['href']
        all_recipes.append('https://cooking.nytimes.com'+link_string)

all_recipes

['https://cooking.nytimes.com//recipes/1021440-jalapeno-jangjorim-with-jammy-eggs',
 'https://cooking.nytimes.com//recipes/1021439-gamja-salad-with-cucumber-carrot-and-red-onion',
 'https://cooking.nytimes.com//recipes/1021485-one-pan-orzo-with-spinach-and-feta',
 'https://cooking.nytimes.com//recipes/1021490-spicy-tamarind-pork-ribs-with-scallions-and-peanuts',
 'https://cooking.nytimes.com//recipes/1021456-lowcountry-okra-soup',
 'https://cooking.nytimes.com//recipes/1021486-eggplant-caponata-pasta-with-ricotta-and-basil',
 'https://cooking.nytimes.com//recipes/1021483-salt-baked-new-potatoes-with-pink-peppercorn-butter',
 'https://cooking.nytimes.com//recipes/1021455-simple-roast-chicken-with-greens-and-bonus-stock',
 'https://cooking.nytimes.com//recipes/1021451-colcannon-with-crispy-leeks',
 'https://cooking.nytimes.com//recipes/1021421-spinach-and-chermoula-pie',
 'https://cooking.nytimes.com//recipes/1021452-one-pan-tuna-white-bean-casserole',
 'https://cooking.nytimes.com//reci

In [165]:
len(all_recipes)

1920

In [172]:
# save list of all recipes to a pickle
import pickle
with open('all_recipes.pickle', 'wb') as to_write:
    pickle.dump(all_recipes, to_write)

In [183]:
recipe_url = 'https://cooking.nytimes.com/recipes/1021319-chile-oil-noodles-with-cilantro'
recipe_response = requests.get(recipe_url)
recipe_response.status_code

200

In [184]:
page = recipe_response.text
soup = BeautifulSoup(page, "lxml")

In [185]:
#topnote
topnote = soup.find('div', {'class': 'topnote'}).find('p').text
print(topnote)

In this 20-minute recipe, a mixture of savory condiments coats bowlfuls of wide noodles chilled slightly by a quick rinse in cool water. While you cook the udon, take the time to prepare the sauce, abundant with contrasting flavors, and the fresh herbs. The sauce can be made in advance, but make sure it’s at room temperature before tossing it with the noodles and the cilantro at the last minute. Substitutions are welcome: Swap in chile crisp in place of the chile oil with crunchy garlic, or scallions in place of garlic chives. Sichuan chile oil brings a citrusy flavor that is hard to replicate, so don’t skip it. It can vary in spice level: For a milder sauce, use only the liquid oil, or add Sichuan peppercorns from the bottom of the oil for extra tingle. Fried shallots are here for texture, but omit them if you use chile crisp.


In [186]:
#tags
tags = soup.find_all('a',{'class':'tag'})
for t in tags:
    print(t.text)

Noodles
Chile Crisp
Chile Oil
Noodle
Sesame Oil
Weekday
Main Course
Vegan
Vegetarian


In [187]:
#ingredients
ingredients = soup.find_all('span',{'class':'ingredient-name'})
for i in ingredients:
    ingredient = i.text.strip()
    print(ingredient)

ounces dried udon noodles
cup chile oil with crunchy garlic
tablespoons pure sesame oil
teaspoons Sichuan chile oil, or to taste
teaspoons soy sauce
cup finely sliced garlic chives or scallions, plus more for garnish
tablespoons store-bought fried shallots, crumbled by hand (optional)
cup finely chopped cilantro (see Note), plus a few sprigs for garnish


In [188]:
#author
author = soup.find('div', {'class': 'nytc---recipebyline---bylinePart'}).find('a').text
print(author)

Judy Kim


In [189]:
#cook_time
cook_time = soup.find('p', {'class': 'cooking-time'}).text
print(cook_time)


30 minutes


In [201]:
#calories
calories = soup.find('span', {'class': 'calorie-count'}).text
print(calories)


AttributeError: 'NoneType' object has no attribute 'text'

In [191]:
#features
features = soup.find('p', {'class': "related-article"}).find_all('a')
for f in features:
    feature =f.text.strip()
    print(feature)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [192]:
#upload_date (if available)
app_json = soup.find('script', {'type': 'application/ld+json'})
app_json = str(app_json)
start = '"uploadDate":"'
upload_date = app_json.split(start)[1][:10]
upload_date

IndexError: list index out of range

In [194]:
#img_date
img = str(soup.find('meta', {'property': 'og:image'}))
img_start = 'images/'
img_date = img.split(img_start)[1][:10]
img_date

'2020/08/09'

In [224]:
#rating count
rating_start = '"ratingCount":'
rating_count = app_json.split(rating_start)[1].split('}')[0]
rating_count

'815'

In [287]:
#avg rating
avg_rating_start = '"ratingValue":'
avg_rating = app_json.split(avg_rating_start)[1].split(',')[0]
avg_rating

'4'

In [289]:
#title
title = soup.find('h1', {'class': 'recipe-title title name'}).text.strip()
print(title)

#content > div.recipe > div > article > header > div.title-container > h1

Creamy Corn Pasta With Basil


In [290]:
#recipe steps
recipe_steps = soup.find_all('ol',{'class':'recipe-steps'})
for r in recipe_steps:
    step = r.text.strip()
    print(step)

Bring a large pot of well-salted water to a boil. Cook pasta until 1 minute shy of al dente, according to the package directions. Drain, reserving 1/2 cup of pasta water.
Meanwhile, heat oil in large sauté pan over medium heat; add scallion whites and a pinch of salt and cook until soft, 3 minutes. Add 1/4 cup water and all but 1/4 cup corn; simmer until corn is heated through and almost tender, 3 to 5 minutes. Add 1/4 teaspoon salt and 1/4 teaspoon pepper, transfer to a blender, and purée mixture until smooth, adding a little extra water if needed to get a thick but pourable texture.
Heat the same skillet over high heat. Add butter and let melt. Add reserved 1/4 cup corn and cook until tender, 1 to 2 minutes. (It’s O.K. if the butter browns; that deepens the flavor.) Add the corn purée and cook for 30 seconds to heat and combine the flavors.
Reduce heat to medium. Add pasta and half the reserved pasta cooking water, tossing to coat. Cook for 1 minute, then add a little more of the pas

In [268]:
#create smaller dataset to test feature extraction 
ten_recipes = recipe_urls.head().append(recipe_urls.tail())
ten_recipes

,0
0,https://cooking.nytimes.com//recipes/1021440-j...
1,https://cooking.nytimes.com//recipes/1021439-g...
2,https://cooking.nytimes.com//recipes/1021485-o...
3,https://cooking.nytimes.com//recipes/1021490-s...
4,https://cooking.nytimes.com//recipes/1021456-l...
1915,https://cooking.nytimes.com//recipes/1015847-p...
1916,https://cooking.nytimes.com//recipes/1015828-m...
1917,https://cooking.nytimes.com//recipes/1015830-m...
1918,https://cooking.nytimes.com//recipes/1015829-m...
1919,https://cooking.nytimes.com//recipes/1015831-m...


In [352]:
def get_recipe_details(recipe_url):
    '''
    Creates a dictionary of the categories of scraped data for each recipe. Returns NaN for unavailable data.

    Parameters
    ----------
    url : the url for a recipe on cooking.nytimes.com

    Returns
    -------
    A dictionary of scraped data for the recipe
    '''
    # request HTML and parse
    recipe_response = requests.get(recipe_url)
    page = recipe_response.text
    soup = BeautifulSoup(page, "lxml")
    
    # title
    try:
        title = soup.find('h1', {'class': 'recipe-title title name'}).text.strip()
    except:
        title = np.NaN
    
    # date of the original image post date
    try:
        img = str(soup.find('meta', {'property': 'og:image'}))
        img_start = 'images/'
        img_date = img.split(img_start)[1][:10]
    except:
        img_date = np.NaN
    
    # upload date
    try:
        app_json = str(soup.find('script', {'type': 'application/ld+json'}))
        start = '"uploadDate":"'
        upload_date = app_json.split(start)[1][:10]
    except:
        upload_date = np.NaN
    
    # instances of article features
    try:    
        features_html = soup.find('p', {'class': "related-article"}).find_all('a')
        features = []
        for f in features_html:
            feature = f.text.strip()
            features.append(feature)
    except:
        features = np.NaN
    
    # calories per serving
    try:
        calories = soup.find('span', {'class': 'calorie-count'}).text
    except:
        calories = np.NaN
    
    # cook time
    try:
        cook_time = soup.find('p', {'class': 'cooking-time'}).text
    except:
        cook_time = np.NaN
    
    #recipe steps
    try:
        recipe_steps_html = soup.find('ol',{'class':'recipe-steps'}).find_all('li')
        recipe_steps = []
        for r in recipe_steps_html:
            step = r.text.strip()
            recipe_steps.append(step)
        num_steps = len(recipe_steps)
    except:
        recipe_steps = np.NaN
        num_steps = np.NaN
        
    # author
    try:
        author = soup.find('div', {'class': 'nytc---recipebyline---bylinePart'}).find('a').text
    except:
        author = np.NaN
    
    # ingredients and number of ingredients  
    try:
        ingredients_html = soup.find_all('span',{'class':'ingredient-name'})
        ingredients = []
        for i in ingredients_html:
            ingredient = i.text.strip()
            ingredients.append(ingredient)
        num_ingredients = len(ingredients)
        #str_ingredients = ",".join(ingredients)
    except:
        ingredients = np.NaN
        num_ingredients = np.NaN
        #str_ingredients = np.NaN
    
    # tags
    try:
        
        tags_html = soup.find_all('a',{'class':'tag'})
        tags = []
        for t in tags_html:
            tags.append(t.text)
    except:
        tags = np.NaN

    # topnote
    try:
        topnote = soup.find('div', {'class': 'topnote'}).find('p').text
    except:
        topnote = np.NaN
        
    # rating count
    try:
        rating_start = '"ratingCount":'
        rating_count = app_json.split(rating_start)[1].split('}')[0]
    except:
        rating_count = np.NaN
        
    # avg rating
    try:
        avg_rating_start = '"ratingValue":'
        avg_rating = app_json.split(avg_rating_start)[1].split(',')[0]
    except:
        avg_rating = np.NaN
    
    # add all features to dict
    headers = ['title','img_date', 'upload_date', 'features', 'calories', 'cook_time', 'recipe_steps', 'num_steps', 'author',
                                     'ingredients','num_ingredients', 'tags', 'topnote', 'rating_count', 'avg_rating']
    recipe_dict = dict(zip(headers, [title, img_date, upload_date, features, calories, cook_time, recipe_steps, num_steps, author,
                                     ingredients, num_ingredients, tags, topnote, rating_count, avg_rating]))
    return(recipe_dict)

    



In [317]:
x = get_recipe_details('https://cooking.nytimes.com/recipes/1021319-chile-oil-noodles-with-cilantro')
x

{'title': 'Chile-Oil Noodles With Cilantro',
 'img_date': '2020/08/09',
 'upload_date': nan,
 'features': nan,
 'calories': nan,
 'cook_time': '30 minutes',
 'recipe_steps': ['Bring a large pot of water to boil and cook noodles according to package instructions, stirring from time to time to prevent them from sticking. Drain well in a colander, then run noodles under cold water until cooled.',
  'Meanwhile, in a large bowl, combine all three oils with the soy sauce and 1/2 cup garlic chives.',
  'Toss cooled noodles into the chile oil mixture. Gently fold in the crumbled fried shallots and chopped cilantro. Divide among four bowls, and top with more garlic chives and cilantro sprigs.'],
 'num_steps': 3,
 'author': 'Judy Kim',
 'ingredients': ['ounces dried udon noodles',
  'cup chile oil with crunchy garlic',
  'tablespoons pure sesame oil',
  'teaspoons Sichuan chile oil, or to taste',
  'teaspoons soy sauce',
  'cup finely sliced garlic chives or scallions, plus more for garnish',
  

In [313]:
recipe_steps_html = soup.find('ol',{'class':'recipe-steps'}).find_all('li')
recipe_steps = []
for r in recipe_steps_html:
    step = r.text.strip()
    print(step)
    recipe_steps.append(step)


Bring a large pot of well-salted water to a boil. Cook pasta until 1 minute shy of al dente, according to the package directions. Drain, reserving 1/2 cup of pasta water.
Meanwhile, heat oil in large sauté pan over medium heat; add scallion whites and a pinch of salt and cook until soft, 3 minutes. Add 1/4 cup water and all but 1/4 cup corn; simmer until corn is heated through and almost tender, 3 to 5 minutes. Add 1/4 teaspoon salt and 1/4 teaspoon pepper, transfer to a blender, and purée mixture until smooth, adding a little extra water if needed to get a thick but pourable texture.
Heat the same skillet over high heat. Add butter and let melt. Add reserved 1/4 cup corn and cook until tender, 1 to 2 minutes. (It’s O.K. if the butter browns; that deepens the flavor.) Add the corn purée and cook for 30 seconds to heat and combine the flavors.
Reduce heat to medium. Add pasta and half the reserved pasta cooking water, tossing to coat. Cook for 1 minute, then add a little more of the pas

In [314]:
len(recipe_steps)

4

In [354]:
list_of_recipe_dicts = []
count = 0

for link in recipe_urls.iloc[603:][0]:
    one_recipe = get_recipe_details(link)
    time.sleep(.5+2*random.random())
    list_of_recipe_dicts.append(one_recipe) #list of dicts to turn into dataframe
    count +=1
    print("Recipes scraped:{}".format(count))


Recipes scraped:1
Recipes scraped:2
Recipes scraped:3
Recipes scraped:4
Recipes scraped:5
Recipes scraped:6
Recipes scraped:7
Recipes scraped:8
Recipes scraped:9
Recipes scraped:10
Recipes scraped:11
Recipes scraped:12
Recipes scraped:13
Recipes scraped:14
Recipes scraped:15
Recipes scraped:16
Recipes scraped:17
Recipes scraped:18
Recipes scraped:19
Recipes scraped:20
Recipes scraped:21
Recipes scraped:22
Recipes scraped:23
Recipes scraped:24
Recipes scraped:25
Recipes scraped:26
Recipes scraped:27
Recipes scraped:28
Recipes scraped:29
Recipes scraped:30
Recipes scraped:31
Recipes scraped:32
Recipes scraped:33
Recipes scraped:34
Recipes scraped:35
Recipes scraped:36
Recipes scraped:37
Recipes scraped:38
Recipes scraped:39
Recipes scraped:40
Recipes scraped:41
Recipes scraped:42
Recipes scraped:43
Recipes scraped:44
Recipes scraped:45
Recipes scraped:46
Recipes scraped:47
Recipes scraped:48
Recipes scraped:49
Recipes scraped:50
Recipes scraped:51
Recipes scraped:52
Recipes scraped:53
Re

Recipes scraped:417
Recipes scraped:418
Recipes scraped:419
Recipes scraped:420
Recipes scraped:421
Recipes scraped:422
Recipes scraped:423
Recipes scraped:424
Recipes scraped:425
Recipes scraped:426
Recipes scraped:427
Recipes scraped:428
Recipes scraped:429
Recipes scraped:430
Recipes scraped:431
Recipes scraped:432
Recipes scraped:433
Recipes scraped:434
Recipes scraped:435
Recipes scraped:436
Recipes scraped:437
Recipes scraped:438
Recipes scraped:439
Recipes scraped:440
Recipes scraped:441
Recipes scraped:442
Recipes scraped:443
Recipes scraped:444
Recipes scraped:445
Recipes scraped:446
Recipes scraped:447
Recipes scraped:448
Recipes scraped:449
Recipes scraped:450
Recipes scraped:451
Recipes scraped:452
Recipes scraped:453
Recipes scraped:454
Recipes scraped:455
Recipes scraped:456
Recipes scraped:457
Recipes scraped:458
Recipes scraped:459
Recipes scraped:460
Recipes scraped:461
Recipes scraped:462
Recipes scraped:463
Recipes scraped:464
Recipes scraped:465
Recipes scraped:466


Recipes scraped:827
Recipes scraped:828
Recipes scraped:829
Recipes scraped:830
Recipes scraped:831
Recipes scraped:832
Recipes scraped:833
Recipes scraped:834
Recipes scraped:835
Recipes scraped:836
Recipes scraped:837
Recipes scraped:838
Recipes scraped:839
Recipes scraped:840
Recipes scraped:841
Recipes scraped:842
Recipes scraped:843
Recipes scraped:844
Recipes scraped:845
Recipes scraped:846
Recipes scraped:847
Recipes scraped:848
Recipes scraped:849
Recipes scraped:850
Recipes scraped:851
Recipes scraped:852
Recipes scraped:853
Recipes scraped:854
Recipes scraped:855
Recipes scraped:856
Recipes scraped:857
Recipes scraped:858
Recipes scraped:859
Recipes scraped:860
Recipes scraped:861
Recipes scraped:862
Recipes scraped:863
Recipes scraped:864
Recipes scraped:865
Recipes scraped:866
Recipes scraped:867
Recipes scraped:868
Recipes scraped:869
Recipes scraped:870
Recipes scraped:871
Recipes scraped:872
Recipes scraped:873
Recipes scraped:874
Recipes scraped:875
Recipes scraped:876


Recipes scraped:1226
Recipes scraped:1227
Recipes scraped:1228
Recipes scraped:1229
Recipes scraped:1230
Recipes scraped:1231
Recipes scraped:1232
Recipes scraped:1233
Recipes scraped:1234
Recipes scraped:1235
Recipes scraped:1236
Recipes scraped:1237
Recipes scraped:1238
Recipes scraped:1239
Recipes scraped:1240
Recipes scraped:1241
Recipes scraped:1242
Recipes scraped:1243
Recipes scraped:1244
Recipes scraped:1245
Recipes scraped:1246
Recipes scraped:1247
Recipes scraped:1248
Recipes scraped:1249
Recipes scraped:1250
Recipes scraped:1251
Recipes scraped:1252
Recipes scraped:1253
Recipes scraped:1254
Recipes scraped:1255
Recipes scraped:1256
Recipes scraped:1257
Recipes scraped:1258
Recipes scraped:1259
Recipes scraped:1260
Recipes scraped:1261
Recipes scraped:1262
Recipes scraped:1263
Recipes scraped:1264
Recipes scraped:1265
Recipes scraped:1266
Recipes scraped:1267
Recipes scraped:1268
Recipes scraped:1269
Recipes scraped:1270
Recipes scraped:1271
Recipes scraped:1272
Recipes scrap

In [355]:
df_second_batch = pd.DataFrame(list_of_recipe_dicts)  
df_second_batch.shape

(1317, 15)

In [338]:
list_of_recipe_dicts

[{'title': 'Jalapeño Jangjorim With Jammy Eggs',
  'img_date': '2020/09/30',
  'upload_date': nan,
  'features': ['A Spread Worthy Of Royalty'],
  'calories': nan,
  'cook_time': 'About 30 minutes',
  'recipe_steps': ['In a medium pot over high heat, bring 6 cups water to a boil. Set up an ice bath in a medium bowl.',
   'Using a spoon, gently lower the eggs into the boiling water, reduce the heat to medium-high and cook at a moderate boil for 8 minutes. Immediately transfer the eggs into the ice bath. Let the eggs cool for 5 minutes, then peel and set aside.',
   'Gently nestle the beef brisket into the pot of hot water; add the onion, scallions, dasima and ginger. Bring to a boil again, then reduce the heat to medium and cook at a gentle boil until the meat is tender, about 1 1/2 hours.',
   'Using a pair of tongs, transfer the meat onto a cutting board and let cool slightly. As the meat cools, strain the broth through a fine-mesh strainer into a large bowl and discard the aromatics.

In [344]:
with open('first_batch.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

In [351]:
recipe_urls.iloc[603:]

,0
603,https://cooking.nytimes.com//recipes/1020029-m...
604,https://cooking.nytimes.com//recipes/1020028-s...
605,https://cooking.nytimes.com//recipes/1020027-r...
606,https://cooking.nytimes.com//recipes/1020026-p...
607,https://cooking.nytimes.com//recipes/1020025-r...
...,...
1915,https://cooking.nytimes.com//recipes/1015847-p...
1916,https://cooking.nytimes.com//recipes/1015828-m...
1917,https://cooking.nytimes.com//recipes/1015830-m...
1918,https://cooking.nytimes.com//recipes/1015829-m...


In [348]:
recipe_urls.to_clipboard(excel=True)

In [357]:
df.shape


(603, 15)

In [359]:
df_second_batch.shape

(1317, 15)

In [361]:
df = df.append(df_second_batch)

In [362]:
df.shape

(1920, 15)

In [363]:
with open('all_scraped_recipes.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)